In [2]:
import pandas as pd

df = pd.read_csv('../data/car-price-prediction-x/train.csv')

print(df.head())
print(df.info())


           model  year motor_type        running wheel    color   type  \
0         toyota  2022     petrol       3000  km  left  skyblue  sedan   
1  mercedes-benz  2014     petrol     132000  km  left    black  sedan   
2            kia  2018     petrol   95000  miles  left    other  sedan   
3  mercedes-benz  2002     petrol  137000  miles  left   golden  sedan   
4  mercedes-benz  2017     petrol     130000  km  left    black  sedan   

      status  motor_volume  price  
0  excellent           2.0  24500  
1  excellent           2.0  25500  
2  excellent           2.0  11700  
3  excellent           3.2  12000  
4       good           2.0  26000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running     

In [3]:
categorical_cols = ['model', 'motor_type', 'wheel', 'color', 'type', 'status']
print(df[categorical_cols].nunique())


model          5
motor_type     5
wheel          1
color         17
type           7
status         5
dtype: int64


In [4]:
categorical_cols = ['model', 'motor_type', 'wheel', 'color', 'type', 'status']
print(df[categorical_cols].nunique())


model          5
motor_type     5
wheel          1
color         17
type           7
status         5
dtype: int64


In [5]:
# ✅ Step 1 — Convert running to numeric KM
def convert_running(val):
    val = val.lower().strip()
    if 'km' in val:
        return float(val.replace('km', '').replace(',', '').strip())
    elif 'miles' in val:
        return float(val.replace('miles', '').replace(',', '').strip()) * 1.60934
    else:
        return 0  # or np.nan

df['running_km'] = df['running'].apply(convert_running)

print(df[['running', 'running_km']].head())


         running  running_km
0       3000  km     3000.00
1     132000  km   132000.00
2   95000  miles   152887.30
3  137000  miles   220479.58
4     130000  km   130000.00


In [6]:
# Now dummy encode
df_encoded = pd.get_dummies(
    df.drop(columns=['running']),  # drop raw text column if not needed
    columns=['model', 'motor_type', 'wheel', 'color', 'type', 'status'],
    drop_first=True
)

df_encoded.to_csv('../data/preprocessed_data.csv', index=False)


In [ ]:
# Save preprocessed dataset

df_encoded = pd.get_dummies(df, columns=['model', 'motor_type', 'wheel', 'color', 'type', 'status'], drop_first=True)
df_encoded.to_csv('../data/preprocessed_data.csv', index=False)


In [7]:
df = pd.read_csv('../data/preprocessed_data.csv')


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['running_km_scaled'] = scaler.fit_transform(df[['running_km']])


In [9]:
X = df[['year', 'motor_volume', 'running_km_scaled']]  # plus dummies
X = X.join(df.drop(columns=['year', 'motor_volume', 'running_km_scaled', 'price']))

y = df['price']

# Split, train, save...
